In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
import copy
%matplotlib inline
import skimage.segmentation
from sklearn.linear_model import LinearRegression
from skimage.color import gray2rgb
import os
from PIL import Image
import json

In [3]:
import shap
import cv2
import matplotlib
from PIL import Image

import shap
import matplotlib
import joblib

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [4]:
#importing required libraries
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

In [6]:
dataframe = pd.read_csv("../xai_thyroid/Data/labels.csv", encoding='utf-8')
for x in dataframe['Image']:
    print(x)

1_1
1_2
2_1
3_1
4_1
5_1
6_1
7_1
8_1
9_1
10_1
11_1
12_1
13_1
14_1
15_1
16_1
17_1
18_1
19_1
20_1
21_1
22_1
23_1
24_1
25_1
26_1
27_1
28_1
29_1
30_1
31_1
32_1
33_1
34_1
35_1
36_1
38_1
39_1
40_1
41_1
42_1
43_1
44_1
45_1
46_1
47_1
48_1
49_1
50_1
51_1
52_1
53_1
54_1
55_1
56_1
57_1
58_1
58_2
59_1
60_1
61_1
62_1
62_2
62_3
63_1
64_1
64_2
64_3
65_1
65_2
66_1
67_1
68_1
69_1
69_2
69_3
70_1
71_1
72_1
73_1
74_1
75_1
75_2
76_1
77_1
78_1
79_1
80_1
80_2
81_1
81_2
82_1
83_1
84_1
85_1
85_2
86_1
87_1
88_1
88_2
89_1
90_1
90_2
91_1
92_1
92_2
92_3
92_4
93_1
93_2
94_1
95_1
96_1
96_2
97_1
97_2
98_1
99_1
99_2
100_1
101_1
102_1
103_1
104_1
105_1
106_1
106_2
106_3
106_4
107_1
108_1
109_1
110_1
111_1
112_1
113_1
114_2
115_1
116_1
117_1
118_1
119_1
120_1
121_1
122_1
122_2
123_1
124_1
125_1
126_1
127_1
128_1
129_1
129_2
130_1
130_2
131_1
132_1
132_2
133_1
134_1
135_1
136_1
137_1
138_1
138_2
139_1
139_2
139_3
140_1
141_1
142_1
142_2
143_1
144_1
145_1
146_1
146_2
147_1
148_1
149_1
150_1
151_1
152_1
153_1
154_1
155_1
15

In [4]:

# read by default 1st sheet of an excel file
# df = pd.read_excel('xai_thyroid/Data/labels.xlsx', header=0)
# df.to_csv('xai_thyroid/Data/labels.csv', index=False)
dataframe = pd.read_csv("../xai_thyroid/Data/labels.csv", encoding='utf-8')
dataframe = dataframe[:480]
# print(dataframe)

dataset = []
for i in range(len(dataframe)):
    if dataframe["Cla1"][i]==0:
        continue
    row_data = []
    
    for row in dataframe:
        row_data.append(dataframe[row][i])
    dataset.append(row_data)

dataset

[['2_1', 1, '2'],
 ['3_1', 2, '4a'],
 ['4_1', 2, '4a'],
 ['5_1', 2, '5'],
 ['6_1', 2, '4b'],
 ['7_1', 2, '4a'],
 ['8_1', 2, '4c'],
 ['9_1', 2, '4b'],
 ['10_1', 2, '4b'],
 ['11_1', 2, '5'],
 ['12_1', 2, '4a'],
 ['13_1', 2, '4a'],
 ['14_1', 1, '2'],
 ['15_1', 2, '4c'],
 ['16_1', 1, '3'],
 ['17_1', 1, '2'],
 ['18_1', 2, '4a'],
 ['19_1', 2, '5'],
 ['20_1', 2, '4b'],
 ['21_1', 2, '4a'],
 ['22_1', 1, '2'],
 ['23_1', 2, '4a'],
 ['24_1', 2, '4b'],
 ['25_1', 1, '3'],
 ['26_1', 2, '4a'],
 ['27_1', 2, '4b'],
 ['28_1', 2, '4b'],
 ['29_1', 2, '5'],
 ['30_1', 2, '4c'],
 ['31_1', 1, '3'],
 ['32_1', 2, '4a'],
 ['33_1', 2, '4a'],
 ['34_1', 2, '4c'],
 ['35_1', 2, '4b'],
 ['36_1', 2, '5'],
 ['38_1', 2, '4b'],
 ['39_1', 2, '4a'],
 ['40_1', 2, '4a'],
 ['41_1', 2, '4b'],
 ['42_1', 2, '4b'],
 ['43_1', 2, '4c'],
 ['44_1', 2, '4a'],
 ['45_1', 2, '4b'],
 ['46_1', 2, '5'],
 ['47_1', 2, '4a'],
 ['48_1', 2, '4b'],
 ['49_1', 2, '4a'],
 ['50_1', 2, '4a'],
 ['51_1', 1, '2'],
 ['52_1', 2, '4a'],
 ['53_1', 2, '4c'],
 [

In [5]:
def print_annotated_image_by_doctor(xml_url,image_path):
    lst=list()
    tree = ET.parse(xml_url)
    root = tree.getroot()
    for child in root:
        if child.tag=='mark':
            for child_child in child:
                if child_child.tag=='svg':
                    lst=json.loads(child_child.text)
    points=lst[0]["points"]
    n=len(lst[0]["points"])
    coordinates=np.zeros((n,2))
    for i in range(n):
        coordinates[i][0]=points[i]['x']
        coordinates[i][1]=points[i]['y']
    image = skimage.io.imread(image_path)
#     print(type(image))
    plt.imshow(image)
    plt.plot("og", markersize=10)  # og:shorthand for green circle
    plt.scatter(coordinates[:, 0], coordinates[:, 1], marker="o", color="yellow", s=20)
    plt.show()
#     print(image.shape)
    return image

In [6]:
def convert_to_single_image(shap_values1):
    a = []
    b = shap_values1.values

    for i in range(len(b)):
        arr1 = []
        for j  in range(len(b[i])):
            arr2 = []
            for k in range(len(b[i][j])):
                arr = np.array([np.array([b[i][j][k][0][0]]), np.array([b[i][j][k][1][0]])])
                arr2.append(arr)
            arr1.append(np.array(arr2))
        a.append(np.array(arr1))
    a = np.array(a)
    
    
    b = shap_values1.data
    a1 = []
    for i in range(len(b)):
        arr1 = []
        for j  in range(len(b[i])):
            arr2 = []
            for k in range(len(b[i][j])):
                arr = np.array([b[i][j][k][0]])
                arr2.append(arr)
            arr1.append(np.array(arr2))
        a1.append(np.array(arr1))
    a1 = np.array(a1)
    
    c = shap_values1
    c.values = a
    c.data = a1
    
    return c

# PB-DCT-SVM Results

In [1]:
it=0

dataframe = pd.read_csv("../xai_thyroid/Data/labels.csv", encoding='utf-8')

dataset = []
for i in range(len(dataframe)):
    if dataframe["Cla1"][i]==0:
        continue
    row_data = []
    
    for row in dataframe:
        row_data.append(dataframe[row][i])
    dataset.append(row_data)

for i in range(0,len(dataset)):
    row = dataset[i]
    filename = row[0]
    if row[0]=="54_1" or row[0]=="127_1" or row[0]=="142_1" or row[0]=="142_2" or row[0]=="166_1" or row[0]=="176_1":
        continue
    if row[0]=="197_1" or row[0]=="203_1" or row[0]=="205_1":
        continue
        
    
        
    print(row[0], " ",)
    if row[1]==1:
        print("Benign")
    if row[1]==2:
        print("Malignant")
    imgno = str(row[0])
    imgno = imgno.split("_")
    xml_url = "../xai_thyroid/Data/archive/" + imgno[0] + ".xml"
    image_path="../xai_thyroid/Data/archive/" + str(row[0]) + ".jpg"
    img = print_annotated_image_by_doctor(xml_url,image_path)
    
    ex_filename = "Results/PB-DCT-SVM blur(28,28) masker Model Results/" + str(row[1]) + "/" + str(row[0]) + '.bz2'
    shap_values1 = joblib.load(filename=ex_filename)
    shap_values1 = convert_to_single_image(shap_values1)
    shap.image_plot(shap_values1/255)
    
    
    
    
    
    
    
    ex_filename = "Results/PB-DCT-SVM inpaint_telea masker Model Results/" + str(row[1]) + "/" + str(row[0]) + '.bz2'
    shap_values1 = joblib.load(filename=ex_filename)
    shap_values1 = convert_to_single_image(shap_values1)
    shap.image_plot(shap_values1/255)

NameError: name 'pd' is not defined